In [1]:
from api import spotify_api
import json
import datetime
import re
from time import gmtime, strftime

settings_file = 'api/settings.txt'
albums_cache_file = 'albums_cache.txt'

max_album_age = 4 #years

In [2]:
access_token = spotify_api.refresh_token(settings_file)


Requesting refresh: 
<Response [200]>
200 OK
BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc
BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc


In [3]:
spotify_request_url = 'https://api.spotify.com/v1/'

next_request = spotify_request_url + 'me/following?type=artist&limit=10'
artists_dict = {}

while(True):
    next_request = next_request + '&access_token=' + access_token.strip()
    artists_json = spotify_api.spotify_request(next_request)

    print('\n')
    artist_items = artists_json['artists']['items']
    for item in artist_items:
        print(item['name']+' -> '+item['id'])
        artists_dict[item['name']] = item['id']

    next_request = artists_json['artists']['next']
    
    print('\n')
    if not next_request:
        print("---END---")
        break


Requesting: https://api.spotify.com/v1/me/following?type=artist&limit=10&access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc
200 OK


Skyharbor -> 0YbvZPP1HufDyWIsR0ei6K
The Algorithm -> 14u4KXVp0iXQil79EpxXGc
Limp Bizkit -> 165ZgPlLkK7bf5bDoFc6Sb
Oh Hiroshima -> 19iNYCVHcMwJMS3NZqhZ59
If These Trees Could Talk -> 2GVzsXcXyU95u2EahzwqN7
Caravan Palace -> 37J1PlAkhRK7yrZUtqaUpQ
Steel Panther -> 3l02WF362j1oHOurzuseBv
Electric Wizard -> 4htjQW3lgIwL6fEJlTOez4
In Flames -> 57ylwQTnFnIhJh4nu4rxCs
Motionless In White -> 6MwPCCR936cYfM1dLsGVnl



Requesting: https://api.spotify.com/v1/me/following?type=artist&after=6MwPCCR936cYfM1dLsGVnl&limit=10&access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tz

In [4]:
albums_list = []
with open(albums_cache_file, 'r') as f:
    for album_id in f.readlines():
        albums_list.append(album_id.strip())

new_albums_dict = {}
new_album_counter = 0
current_year = datetime.datetime.now().year

with open(albums_cache_file, 'a') as f:
    for artist in artists_dict:
        artist_id = artists_dict[artist]
        print('\nArtist: ' + artist)

        next_request = spotify_request_url + 'artists/'+artist_id+'/albums?limit=50&album_type=album,single&market=PL'

        while(True):
            next_request = next_request + '&access_token=' + access_token.strip()
            albums_json = spotify_api.spotify_request(next_request)

            album_items = albums_json['items']
            for item in album_items:
                print(item['name']+' -> '+item['release_date'])
                if item['id'] not in albums_list:
                    if current_year - int(item['release_date'][:4]) < max_album_age:
                        new_albums_dict[item['id']] = (artist, item['name'], item['release_date'])
                        f.write(item['id']+'\n')
                        new_album_counter = new_album_counter + 1

            next_request = albums_json['next']
            if not next_request:
                break
                
print('New albums: ' + str(new_album_counter))


Artist: Skyharbor

Requesting: https://api.spotify.com/v1/artists/0YbvZPP1HufDyWIsR0ei6K/albums?limit=50&album_type=album,single&market=PL&access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc
200 OK
Guiding Lights -> 2014-11-10
Blinding White Noise: Illusion & Chaos -> 2012-04-23
Chemical Hands -> 2016-09-23
Blind Side -> 2016-06-09
Out of Time -> 2015-08-25
Evolution -> 2014-06-12

Artist: The Algorithm

Requesting: https://api.spotify.com/v1/artists/14u4KXVp0iXQil79EpxXGc/albums?limit=50&album_type=album,single&market=PL&access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZ

In [5]:
new_albums_dict

{'0iobawuf3V6i1WiUy084uX': ('Linkin Park',
  'Darker Than The Light That Never Bleeds (Chester Forever Steve Aoki Remix)',
  '2017-09-08'),
 '0zcNtUZ2oEpRmiDuWBFvcX': ('Linkin Park',
  'One More Light Live',
  '2017-12-15'),
 '2kizlRv4J1gO7nXZ7kzJdk': ('Linkin Park',
  'Heavy (feat. Kiiara)',
  '2017-02-16'),
 '2zLPdvAfc3K3dkZwukAP3V': ('Linkin Park',
  'Heavy (feat. Kiiara) [Disero Remix]',
  '2017-04-28'),
 '4pRPbwQlthjHvMs6m09phN': ('Linkin Park',
  'Heavy (feat. Kiiara) [Nicky Romero Remix]',
  '2017-04-21'),
 '51WStYbzUKG6O0XZd5Ta9x': ('Linkin Park',
  'One More Light (Steve Aoki Chester Forever Remix)',
  '2017-10-25'),
 '5Eevxp2BCbWq25ZdiXRwYd': ('Linkin Park', 'One More Light', '2017-05-19'),
 '76OiIDD7aFE3f2WJ1a1Z21': ('Linkin Park',
  'Sharp Edges (One More Light Live)',
  '2017-12-01'),
 '79nYut68HnVok4ODuoD44e': ('Linkin Park',
  'Crawling (One More Light Live)',
  '2017-11-15')}

In [6]:
tracks_counter = 0
new_tracks = []

for album_id in new_albums_dict:
    next_request = spotify_request_url + 'albums/'+album_id+'/tracks?limit=50&access_token=' + access_token.strip()
    tracks_json = spotify_api.spotify_request(next_request)
    
    tracks_items = tracks_json['items']
    for item in tracks_items:
        print(item['name'] + ' -> ' + item['uri'])
        new_tracks.append(item['uri'].strip())
        tracks_counter = tracks_counter + 1
        
print('New tracks: ' + str(tracks_counter))
# GET https://api.spotify.com/v1/albums/{id}/tracks


Requesting: https://api.spotify.com/v1/albums/0zcNtUZ2oEpRmiDuWBFvcX/tracks?limit=50&access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc
200 OK
Talking to Myself - One More Light Live -> spotify:track:1DVXrmuh4umsD96KRTsGHl
Burn It Down - One More Light Live -> spotify:track:0zKs47fghFBtrEpxyUR1Wz
Battle Symphony - One More Light Live -> spotify:track:7mXJEniTl79NSO50M2HjBU
New Divide - One More Light Live -> spotify:track:6dWGJ5acTiipYqVpNyOTKd
Invisible - One More Light Live -> spotify:track:5BXGP4IsPuXSVNQYZLQCuD
Nobody Can Save Me - One More Light Live -> spotify:track:6LYZAzd9FoPEecenhggMsy
One More Light - One More Light Live -> spotify:track:1r8qywTSOGQvFxQnzcyiBj
Crawling - One More Light Live -> spotify:track:0w4ZwKHHEkXtvHMNUjFPdB
Leave Out All the Rest - One More L

In [8]:
next_request = 'https://api.spotify.com/v1/me/playlists?access_token=' + access_token.strip()
playlists_json = spotify_api.spotify_request(next_request)

my_playlist = '5QtunQsOyazjCStL8KNz3U'

playlists_items = playlists_json['items']
for item in playlists_items:
    print(item['name'] + ' -> ' + item['id'])
    if item['name'].strip() == 'Music News (Bednar)':
        my_playlist = item['id'].strip()
        
print('\nMy playlist ID: ' + my_playlist)


Requesting: https://api.spotify.com/v1/me/playlists?access_token=BQDxSGp66afKoy9-BqKSi0bTrT7Y8IzgKoMer6TqCaTwa_-aIFJF36I5aQopp9RpeALMbnU8rsFto1OJWlcqddj3DhrkXnMTFTn2W3f65h9j__D8plS3cK4Ss_tzrMHLsREaiY0MaiRYdLxyljfRUXeqX09juKtC5I1qUOWpculWToffYc0Td4HzxvpYt_dfIqDoRgoMZmA9j7BPfR4uC5okZ8Uj1Ud-vc4gQMxSbJmFkvCMEYVzZhIubWc
200 OK
Music News (Bednar) -> 5QtunQsOyazjCStL8KNz3U
Cheerful raps -> 3IG5YeyDxr2WvmtxoABXco
Release Radar -> 37i9dQZEVXbidQSqNh7fI2
N'to - Trauma (Worakls Remix) -> 1R3pob1JtA1rkCfujHBsDV
Instrumental Madness -> 37i9dQZF1DWUk47CLxI4Uo
Discover Weekly -> 37i9dQZEVXcIleIODRmV9m
Liked from Radio -> 5XXEmY7K7glcrIHp28zJ8z

My playlist ID: 5QtunQsOyazjCStL8KNz3U


In [9]:
import requests

max_tracks_per_request = 10
url = 'https://api.spotify.com/v1/users/piotr.bednarski/playlists/' + my_playlist + '/tracks?uris='
headers = {
    'Authorization' : 'Bearer ' + access_token.strip(),
    'Content-Type' : 'application/json'
}
url_full = url
for i in range(len(new_tracks)):
    #print(new_tracks[i])
    url_full = url_full + new_tracks[i] + ','
    
    if (i%max_tracks_per_request == max_tracks_per_request-1) or (i == len(new_tracks)-1):
        url_full = url_full[:-1]
        #print("Request!")
        r = requests.post(url_full, headers=headers)
        #print(r)
        print(r.status_code, r.reason)
        url_full = url

201 Created
201 Created
201 Created
201 Created


In [15]:
with open('feed_stub.xml', 'r') as fs:
    feed_stub=fs.read()

In [18]:
title = strftime("%Y-%m-%d %H:%M Spotify newest releases", gmtime())
feed = re.sub('TITLE', title, feed_stub, flags=(re.MULTILINE | re.DOTALL))

In [19]:
description = 'New releases:\n'
for item in new_albums_dict:
    description = description + '\n' + new_albums_dict[item][0]+ ' - ' + new_albums_dict[item][1]
print(description)
feed = re.sub('DESCRIPTION', description, feed, flags=(re.MULTILINE | re.DOTALL))

New releases:

Linkin Park - One More Light Live
Linkin Park - One More Light
Linkin Park - Sharp Edges (One More Light Live)
Linkin Park - Crawling (One More Light Live)
Linkin Park - One More Light (Steve Aoki Chester Forever Remix)
Linkin Park - Darker Than The Light That Never Bleeds (Chester Forever Steve Aoki Remix)
Linkin Park - Heavy (feat. Kiiara) [Disero Remix]
Linkin Park - Heavy (feat. Kiiara) [Nicky Romero Remix]
Linkin Park - Heavy (feat. Kiiara)


In [20]:
with open('feed.xml', 'w', encoding='utf-8') as f:
    f.write(feed)